In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [2]:
#Load the Image Data

train_data_dir = "E:\\MSc DataAnalytics\\Learning\\Semester3\\Thesis\\RICFiles\\DataSet\\shopee-product-matching\\train_images\\"
test_data_dir = "E:\\MSc DataAnalytics\\Learning\\Semester3\\Thesis\\RICFiles\\DataSet\\shopee-product-matching\\test_images\\"

#load the description
df_train_data = pd.read_csv('E:\\MSc DataAnalytics\\Learning\\Semester3\\Thesis\\RICFiles\DataSet\\shopee-product-matching\\train.csv')
df_test_data = pd.read_csv('E:\\MSc DataAnalytics\\Learning\\Semester3\\Thesis\\RICFiles\DataSet\\shopee-product-matching\\test.csv')

df_train_data['image_path'] = train_data_dir + (df_train_data['image'])

classes = np.unique(df_train_data['label_group'])
class_to_idx = {classes[i]: i for i in range(len(classes))}

In [3]:
#Creating the class for the shopee data set
class ShopeeDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.classes = np.unique(dataframe['label_group'])
        self.class_to_idx = {self.classes[i]: i for i in range(len(self.classes))}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        img = Image.open(img_path).convert('RGB')
        label = self.class_to_idx[self.dataframe.iloc[idx]['label_group']]

        if self.transform:
            img = self.transform(img)

        return img, label


In [4]:
#transformation for network training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
dataset = ShopeeDataset(df_train_data, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
#loading pretrained ResNet18

model = torchvision.models.resnet18(pretrained=True)

num_classes = len(classes)
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

C:\Users\dskme\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dskme\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
num_epochs = 15

# TRAINING sET
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    print('start trainig epoch .{}'.format(epoch))
    model.train()
    #print("inaise")
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        print()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_acc += torch.sum(preds == labels.data)

    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)

    print('Epoch: {} \t Training Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(
        epoch+1, train_loss, train_acc))
    
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')

start trainig epoch .0























































































































































































































Epoch: 1 	 Training Loss: 9.405019 	 Training Accuracy: 0.030766
Epoch [1/15], Validation Accuracy: 4.23%
start trainig epoch .1























































































































































































































Epoch: 2 	 Training Loss: 7.566181 	 Training Accuracy: 0.132555
Epoch [2/15], Validation Accuracy: 19.40%
start trainig epoch .2























































































































































































































Epoch: 3 	 Training Loss: 4.986526 	 Training Accuracy: 0.348723
Epoch [3

In [8]:
#Restoring the trained path
torch.save(model.state_dict(), 'resnet18_trained.pth')